<a href="https://colab.research.google.com/github/A01706648/TC3006C_Deliver/blob/main/m1/M1_Delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **General Questions:**

**By industry, what is the composition of US public firms in terms of firm size, sales performance and profitability?**

**size:**

We shall consider the firm's total assert, minus the firm's total totalliabilities

But since most of the US firm is in a very high levarage,
We may only need to use firm's total assert as its real size


**Sale performance:**

We shall consdier firm's revenue

**Profitability**

We shall consider the profit margine of the firm


**Why some firms are more profitable than others?**

**Which factors/variables from financial statements are related to stock returns?**


#**Specific Questions:**



In [5]:
isGoogleColab = False

if(isGoogleColab):
    from google.colab import drive
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
industry_dict = {'Accommodation and Food Services':'Services',
       'Administrative and Support and Waste Management and Remediation Services':'Services',
       'Agriculture, Forestry, Fishing and Hunting':'Agriculture',
       'Arts, Entertainment, and Recreation':'Services', 
       'Construction':'Construction',
       'Educational Services':'Services', 
       'Finance and Insurance':'Finance',
       'Health Care and Social Assistance':'Services', 
       'Information':'Services',
       'Manufacturing':'Manufacturing', 
       'Mining, Quarrying, and Oil and Gas Extraction':'Mining',
       'Other Services (except Public Administration)':'Services',
       'Professional, Scientific, and Technical Services':'Services',
       'Real Estate and Rental and Leasing':'Finance', 
       'Retail Trade':'Comercial',
       'Transportation and Warehousing':'Services', 
       'Utilities':'Energy', 
       'Wholesale Trade':'Comercial'}

In [6]:
if(isGoogleColab):
    drive.mount("/content/gdrive") 
    %cd "/content/gdrive/MyDrive/Machine Learning/deliver/M1"
df = pd.read_csv('us2022q2a.csv')
df_firm = pd.read_csv('usfirms2022.csv')

In [7]:
df_more = df.copy()

#put industry column
firm_list = df['firm'].unique()
firm_list = firm_list.tolist()
firm_list.remove(np.nan)
df_firm_more = df_firm.copy()
df_firm_more['industry'] = df_firm_more['Sector NAICS\nlevel 1']
df_firm_more = df_firm_more.replace({'industry': industry_dict})

firm_industry_dic = {}

for firm in firm_list:
  if (len((df_firm_more[df_firm_more['Ticker'] == firm]['industry']).values.tolist()) > 0):
    firm_industry_dic[firm] = ((df_firm_more[df_firm_more['Ticker'] == firm]['industry']).values.tolist())[0]
  else:
    firm_industry_dic[firm] = np.nan

df_more['industry'] = df_more['firm']
df_more = df_more.replace({'industry': firm_industry_dic})


In [8]:


#Calculate bookvalue
df_more['bookvalue'] = df_more['totalassets'] - df_more['totalliabilities']

#Calculate Market Value
df_more['marketvalue'] = df_more['originalprice'] * df_more['sharesoutstanding']

#Calculate Operating profit margin
df_more['op_profit'] = df_more['revenue'] - df_more['cogs'] - df_more['sgae'] - df_more['otheropexp']
df_more['op_profit_margin'] = df_more['op_profit'] / df_more['revenue']
df_more['net_income'] = df_more['op_profit'] - df_more['incometax'] - df_more['finexp'] + df_more['extraincome']
df_more['profit_margin'] = df_more['net_income'] / df_more['revenue']




In [9]:
df_more.head()

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,fiscalmonth,year,cto,industry,bookvalue,marketvalue,op_profit,op_profit_margin,net_income,profit_margin
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2000.0,1.0,Manufacturing,NaN,4.700800e+07,NaN,NaN,NaN,NaN
1,A,2000q2,2485000.0,1261000.0,1010000.0,0.0,42000.0,0.0,90000.0,7321000.0,...,6.0,2000.0,2.0,Manufacturing,4642000.0,3.335506e+07,214000.0,0.086117,166000.0,0.066801
2,A,2000q3,2670000.0,1369000.0,1091000.0,0.0,28000.0,0.0,83000.0,7827000.0,...,9.0,2000.0,3.0,Manufacturing,4902000.0,2.216940e+07,210000.0,0.078652,155000.0,0.058052
3,A,2000q4,3372000.0,1732000.0,1182000.0,0.0,10000.0,0.0,163000.0,8425000.0,...,12.0,2000.0,4.0,Manufacturing,5265000.0,2.498606e+07,458000.0,0.135824,305000.0,0.090451
4,A,2001q1,2841000.0,1449000.0,1113000.0,0.0,-6000.0,0.0,119000.0,9208000.0,...,3.0,2001.0,1.0,Manufacturing,5541000.0,1.403653e+07,279000.0,0.098205,154000.0,0.054206


In [11]:
industry_list = df_more['industry'].unique()
industry_list = industry_list.tolist()
del industry_list[-1]
del industry_list[-1]
industry_list

['Manufacturing',
 'Finance',
 'Services',
 'Comercial',
 'Energy',
 'Construction',
 'Agriculture',
 'Mining']

In [12]:
#Show how many firms by industry there are in the sample
industry_firm_num = []
for industry in industry_list:
  industry_firm_num.append(len(df_more[df_more['industry'] == industry]['firm'].unique()))

df_industry_firm_num = pd.DataFrame({'Industry':industry_list,'firm num':industry_firm_num})
df_industry_firm_num

,Industry,firm num
0,Manufacturing,1565
1,Finance,769
2,Services,792
3,Comercial,231
4,Energy,77
5,Construction,45
6,Agriculture,16
7,Mining,103


**For each industry (and for all industries),**

**what can you say about the typical firm size in terms of market value and book value?**

We analyze the last quater of the market value and book value of all the firms for each industry.

We can see the dispersion in the box plot below.

They are very dispersive.

So we are going to use the median value of the market value and book value,

to indicate the typical firm size.

**How much these variables change within each industry?**

**How firm size (in market value) is distributed?**

In [ ]:
df_final_size = df_more.dropna()
df_final_size = df_final_size.groupby('firm').last()
df_final_size

In [14]:
df_final_size.groupby('industry').median()[['bookvalue', 'marketvalue']]

,bookvalue,marketvalue
industry,,
Agriculture,1068506.5,1.264045e+06
Comercial,565853.0,1.438305e+06
Construction,998146.5,1.900822e+06
Energy,2845900.0,6.040649e+06
Finance,1038181.0,1.785152e+06
Manufacturing,229197.5,5.913003e+05
Mining,679456.0,1.042405e+06
Services,443680.5,1.828864e+06


In [17]:
df_final_size[df_final_size['industry'] == 'Manufacturing']['bookvalue'].describe()

count    1.530000e+03
mean     2.308925e+06
std      9.581465e+06
min     -1.479100e+07
25%      4.616361e+04
50%      2.291975e+05
75%      9.012775e+05
max      1.845080e+08
Name: bookvalue, dtype: float64

In [15]:
px.box(df_final_size, x='industry', y='bookvalue', title='Book Value of all industries')

In [16]:
px.box(df_final_size, x='industry', y='marketvalue', title='Market Value of all industries')

For each industry (and for all industries), 

what can you say about profit margin of firms? 

show 

a) descriptive statistics of profit margin and 

b) plot(s) to illustrate how profit margin changes across industries.

In [85]:
px.box(df_final_size, x='industry', y='profit_margin', title='Profit Margin of all industries')

Which are the biggest 10 US firms in terms of market value and how far they are from the typical size of a US firm?

In [93]:
for industry in industry_list:
  df_temp = df_final_size[df_final_size['industry'] == industry].sort_values(['marketvalue'],ascending=False)[['industry', 'marketvalue']].head(10)
  df_temp.style